To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use a special ORPO style dataset from [recipe-research](https://huggingface.co/datasets/reciperesearch/dolphin-sft-v0.1-preference).

You need at least 3 columns:
* Instruction
* Accepted
* Rejected

For example:
* Instruction: "What is 2+2?"
* Accepted: "The answer is 4"
* Rejected: "The answer is 5"

The goal of ORPO is to penalize the "rejected" samples, and increase the likelihood of "accepted" samples. [recipe-research](https://huggingface.co/datasets/reciperesearch/dolphin-sft-v0.1-preference) essentially used Mistral to generate the "rejected" responses, and used GPT-4 to generated the "accepted" responses.

In [4]:
from unsloth.chat_templates import get_chat_template
from datasets import load_from_disk

In [5]:
# Setup ChatML format
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml"
)

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


In [6]:
dataset_raw = load_from_disk("/content/orpo")

In [7]:
# Print an example row
print(f"""
## Question:
{dataset_raw["train"][0]["search_term"]}

## Chosen:
{dataset_raw["train"][0]["chosen"]}

## Rejected:
{dataset_raw["train"][0]["rejected"]}
""")


## Question:
vilket vin till currykyckling

## Chosen:
<think>Användaren verkar leta efter ett vin som passar till currykyckling. Jag tolkar detta som att användaren söker en matrekommendation där vinets egenskaper ska matcha rättens smakprofil. Jag väljer därför att applicera kategorifiltret för vin, lägga till information om currykycklingens smakprofil och markera att det är en matrekommendation.</think><answer>```json
{"allowed": true, "route": "Food recommendation", "category": "Vin", "taste_symbols": "Fågel", "dish_info": {"description": "Currykyckling är en smakrik rätt där kyckling är huvudingrediensen, ofta med en krämig konsistens från kokosmjölk eller grädde. Den har en balanserad hetta och en aromatisk profil från curry och andra kryddor. Rätten är vanligtvis umamirik med en lätt sötma.", "name": "currykyckling"}}
```</answer>
      
## Rejected:
<think>The search query asks for a wine recommendation to pair with chicken curry. This falls under the "Food recommendation" rou

### Prompt Setup

In [8]:
list_of_categories = "Absint; Akvavit; Akvavit & Kryddat brännvin; Ale; Ale - Mild; Alkoholfritt; Alkoholfritt - Ale; Alkoholfritt - Amerikansk pale ale (APA); Alkoholfritt - Andra juldrycker; Alkoholfritt - Aperitif & Bitter; Alkoholfritt - Avec; Alkoholfritt - Belgisk ljus ale/Blonde; Alkoholfritt - Blanddryck; Alkoholfritt - Brown ale; Alkoholfritt - Brune/Dubbel; Alkoholfritt - Bärmust; Alkoholfritt - Cider; Alkoholfritt - Cider & Blanddryck; Alkoholfritt - Dortmunder och helles; Alkoholfritt - Drinkar & Cocktail; Alkoholfritt - Druvmust; Alkoholfritt - Engelsk pale ale och bitter; Alkoholfritt - Glögg; Alkoholfritt - Gose och lichtenhainer; Alkoholfritt - Hefeweizen; Alkoholfritt - India pale ale (IPA); Alkoholfritt - Internationell stil; Alkoholfritt - Ljus lager; Alkoholfritt - Mellanmörk & Mörk lager; Alkoholfritt - Modern stil/India pale lager; Alkoholfritt - Mousserande rosé; Alkoholfritt - Mousserande vitt; Alkoholfritt - Mousserande övrigt; Alkoholfritt - Must; Alkoholfritt - New England IPA/Hazy IPA; Alkoholfritt - Pilsner - tjeckisk Stil; Alkoholfritt - Pilsner - tysk Stil; Alkoholfritt - Rosévin; Alkoholfritt - Rött vin; Alkoholfritt - Schwarzbier; Alkoholfritt - Smaksatt/kryddad öl; Alkoholfritt - Snaps; Alkoholfritt - Strong ale; Alkoholfritt - Syrlig öl; Alkoholfritt - Torr porter och stout; Alkoholfritt - Veteöl; Alkoholfritt - Vitt vin; Alkoholfritt - Witbier; Alkoholfritt - Äppelmust; Alkoholfritt - Öl; Alkoholfritt - Övrig veteöl; Altbier; Amber ale; Amerikansk pale ale (APA); Amontillado; Anissprit; Annan Madeira; Annan likör; Annan sherry; Annan whisky; Annat rött portvin; Aperitifer; Arak; Armagnac; Arrak; Barley wine; Belgisk ljus ale/Blonde; Berliner weisse; Bitter; Black IPA; Blanddryck; Blanddryck - Söt; Blanddryck - Torr; Blandning av olika öltyper; Blended whisky; Bourbon; Brandy; Brown ale; Brune/Dubbel; Brännvin; Bual/boal; Bärlikör; California common; Calvados; Calvados VS; Calvados VSOP; Calvados XO; Chokladlikör; Cider; Cider & blanddryck; Cider - Söt; Cider - Torr/halvtorr; Cognac; Cognac VS; Cognac VSOP; Cognac XO; Cornwhisky; Cream sherry; Domfrontais; Dortmunder och helles; Drinkar & Cocktails; Druvsprit; Dubbelbock; Dunkel; Dunkelweizen; Engelsk pale ale och bitter; Extra special bitter; Fino; Flamländsk röd och brun öl; Frukt & Druvsprit; Frukt- och bärlambic; Fruktlikör; Fruktsprit; Fruktvin; Genever; Gin; Glögg och Glühwein; Golden/Blond ale; Gose och lichtenhainer; Grainwhisky; Grappa; Gräddlikör; Gueuze; Hefeweizen; Hopfenweizen; Imperial porter och stout; Imperial/Dubbel IPA; India pale ale (IPA); Irländsk ale; Isbock; Kaffelikör; Kristallweizen; Kryddat brännvin; Kryddlikör; Kölsch-stil; Lambic; Likör; Ljus bocköl; Ljus lager; Ljus lager - Internationell stil; Ljus lager - Modern stil/India Pale Lager; Ljus rom; Ljus stark belgisk ale/Trippel; Malmsey/malvasia; Maltsprit; Maltwhisky; Manzanilla; Marc; Mellanmörk & Mörk lager; Mezcal; Mixpack öl; Mjöd; Montilla; Mousserande vin; Mousserande vin - Blått ; Mousserande vin - Fruktvin; Mousserande vin - Halvtorrt vitt; Mousserande vin - Rosé; Mousserande vin - Rött; Mousserande vin - Smaksatt; Mousserande vin - Sött; Mousserande vin - Torrt vitt; Märzen och wienerstil; Mörk rom & Lagrad sockerrörssprit; Mörk stark belgisk ale; New England IPA/Hazy IPA; Nötlikör; Old ale; Oloroso; Ouzo; Palo cortado; Pastis; Pedro ximénez (PX); Pilsner - tjeckisk stil; Pilsner - tysk stil; Polotmavý; Porter & Stout; Portvin; Portvin - Colheita; Portvin - Late bottled vintage (LBV); Portvin - Ruby; Portvin - Tawny; Portvin - Vintage; Portvin - Vitt portvin; Punsch; Raki; Rom & Lagrad sockerrörssprit; Rosévin; Rosévin - Friskt & Bärigt; Rosévin - Fruktigt & Smakrikt; Ryewhisky; Rököl; Rött vin; Rött vin - Fruktigt & Smakrikt; Rött vin - Kryddigt & Mustigt; Rött vin - Mjukt & Bärigt; Rött vin - Stramt & Nyanserat; Rött vin - Sött; Saison; Sake; Sangria; Schwarzbier; Sercial; Session IPA; Shochu/Soju; Skotsk ale; Smaksatt Soju/Shochu; Smaksatt sprit; Smaksatt sprit av rom; Smaksatt vin; Smaksatt vin & fruktvin; Smaksatt vodka; Smaksatt/kryddad öl; Sprit; Starkare lager; Starkvin; Strong ale; Syrlig öl; Söt porter och stout; Sött fruktvin; Tequila; Tequila & Mezcal; Torr porter och stout; Torrt fruktvin; Verdelho; Vermouth; Vermouth rosé; Vermouth röd söt; Vermouth vit söt; Vermouth vit torr; Veteöl; Vin; Vinlåda; Vitt vin; Vitt vin - Druvigt & Blommigt; Vitt vin - Friskt & Fruktigt; Vitt vin - Fylligt & Smakrikt; Vitt vin - Lätt & Avrundat; Vitt vin - Sött; Vodka; Weizenbock; Wheatwhisky; Whisky; Witbier; Zwickel, keller- och landbier; Ägglikör; Åldersbetecknad calvados; Åldersbetecknad cognac; Årgångsbetecknad calvados; Årgångsbetecknad cognac; Öl; Örtlikör; Övrig spontanjäst öl; Övrig syrlig öl; Övrig veteöl; Övrigt starkvin"

In [10]:
from datetime import datetime

reasoning_start = "<think>"
reasoning_end   = "</think>"
solution_start = "<answer>"
solution_end = "</answer>"

REASONING_TAG = "think"
ANSWER_TAG = "answer"

system_prompt = f"""
# Instruktioner

Assistent fungerar som ett centralt nav i Systembolagets sökfunktion. Genom att noggrant analysera och bearbeta användarens sökningar till API-filter, hjälper Assistents expertis användaren att hitta precis det de letar efter.

För att garantera tydlighet i bedömningen ska varje svar inledas med en redogörelse för tankeprocessen, placerad inom `<think>`-taggar, följt av det slutgiltiga API-anropet i JSON-format inom `<answer>`-taggar. Båda delarna kommer att användas för att informera och utbilda inom området.

## Viktig information om API:et:
- **category:** Anger den kategori eller de kategorier som användaren söker efter. Möjliga värden: <list_of_categories>{list_of_categories}</list_of_categories>.
- **origin:** Anger produktens ursprung (land, region eller ort). Specificera så exakt som möjligt och korrigera stavningen om nödvändigt. Använd alltid den officiella benämningen.
- **allowed:** Indikerar om sökningen är tillåten för Systembolaget. Har en sökning ett en vag referens till pris, kvalitet eller alkoholnivå ska den alltid flaggas upp.
- **moderaton:** Om sökningen är avvikande inkludera anledningen med flaggor såsom:
  - Moderation (för olämpligt eller irrelevant innehåll)
  - Price (för vaga prisbeskrivningar)
  - Alcohol Level (för vaga referenser till alkoholhalt)
  - Quality (för vaga referenser till kvalitet)

  Observera:

- **Range-parametrar:** Måste ha både ett `min`- och ett `max`-värde. Är det en exakt siffra som anges kan den sättas som både min och max.
  - **price:** Pris i svenska kronor
  - **volume:** Volym i milliliter
  - **alcohol_level:** Alkholhalt i procent
  - **sugar_content:** Sockerhalt i gram per 100 ml
  - **product_launch:** Datumintervall för lansering. Format YYYYmmdd

- **Smakbeskrivningar:** Dessa anges på ett intervall från 1 till 12 (där högre värde innebär mer intensitet). Exempelvis betyder ett värde mellan 1-2 för Sweetness att vinet är torrt. Rekommenderat är att använda intervall på mellan 3-6 steg för att säkerställa att produkter finns, med undantag för "torrt" (alltid 1-2). Ska alltid ha `min` och `max`
- Tillgängliga smakbeskrivningar:
  - **sweetness:** Sötma eller torrhet. Ett torrt vitt vin är t.ex. 1-2, en halvtorr cider 3-5 och en öl med hög sötma 8-12
  - **body:** Dryckens fyllighet eller lätthet. Ett fylligt vitt vin är t.ex. 6-12, en lätt rödvin mellan 1-4 och en medelfyllig öl mellan 5-8.
  - **fruit_acid:** Syrlighet. Ett friskt vitt vin är t.ex. 9-12
  - **roughness:** Dryckens strävhet eller mjukhet (endast för vin). Ett stramt rödvin är t.ex. 8-12 medan ett mjukt är mellan 1-5.
  - **bitterness:** Hur bitter drycken är (endast för öl). Låg beska är t.ex. 1-4 och mycket beska mellan 8-12.
- **smokiness:** Beskriver hur rökig en spritsort är. Möjliga värden: `Ingen`, `Liten`, `Tydlig`, `Påtaglig`.
- **cask_taste:** Beskriver om drycken har fatkaraktär eller inte - true eller false.

- **Andra filterparametrar**:
  - **assortment:** Beskriver vilken del av sortimentet produkten tillhör. Kan vara `Fast sortiment`, `Tillfälligt sortiment`, `Ordervaror`, `Webblanseringar`, `Lokalt & Småskaligt` samt `Säsong`.
  - **other_selections:** Information kan läggas in här om en användare söker efter `Glutenfri`, `Koscher`, `Naturvin` eller `Vegansk`
  - **packaging:** Vad för typ av förpackning användaren söker efter. Kan vara `Box`, `Burk`, `Fat`, `Glasflaska`, `Lättare glasflaska`, `Pappförpackning`, `PET-flaska`, `Påse`, `Retuglas`
  - **grape:** Vilken eller vilka druvor användaren söker efter.
  - **vintage:** Specifika årgångar, t.ex. `2001`
  - **news:** Tagg för att visa att en produkt är ny. Sätts till `Nyhet` om så är fallet.
  - **new_arrival_type:** Beskriver olika typer av nyinkomna produkter. Kan vara: `Nytt idag`, `Nytt senaste veckan`, `Nytt senaste månaden`, `Nytt senaste 3 månader`.
  - **upcoming_launch:** Om en användare söker efter ett specifikt lanseringsdatum specificeras datumet med formatet `YYYYmmdd`
  - **text_query:** Används när de andra filtervalen inte helt fångar det användaren söker. Är applicerbart på smakbeskrivningar, produktreferenser och liknande.
  - **ethical_label:** Om en användare söker efter en dryck som är antingen `Fairtrade` eller `Fair for life`.
  - **sustainable_choices:** Om användaren söker efter produkter som är antingen `Ekologiskt` eller `Hållbara drycker`.
  - **climate_footprint:** Beskriver produktens klimatavtryck som antingen `Lägre`, `Medel` eller `Högre`.
  - **extracted_dish_info:** Innehåller specifik information om maträtter som användaren söker. Innehåller alltid två delar: själva namnet och en beskrivning av rättens karaktäristik. Det måste vara en JSON med nycklarna `name` och `description`.
  - **route:** Beskriver sökningstypen. Möjliga värden: `Food recommendation`, `Filter search`, `Irrelevant`, `Product reference`.

- **Notera att butiksreferenser inte stöds och ska därför filtreras bort.**
- **Om flera filter är relevanta, ange dem i samma parameter separerade med kommatecken.**


**Dagens datum är {datetime.now().strftime("%Y-%m-%d")}**

## Viktig information om svarsformatet:
Varje svar ska innehålla två delar:
1. **Tankeprocess (Resonemang):** Placeras inom `<think>`-taggar. Använd så mycket utrymme som behövs för att komma fram till rätt svar. När du tror att du är klar bör du stanna upp och sägaa "vänta, jag måste tänka lite till".
2. **API-anrop:** Placeras inom `<answer>`-taggar och ska vara formaterat som giltig JSON.

## Exempel (använd dessa som inspiration)

### Exempel 1:
- **Sökning:** Franskt vin med smak av viol
- **Svar:** <think>Okej, användaren verkar leta efter ett franskt vin med en violkaraktär. Jag väljer därför att sätta specificera text queryn "viol" utöver land- och kategorivalen.</think><answer>```json{{"text_query": "viol", "route": "Filter search", "origin": "Frankrike", "category": "Vin", "Allowed": true}}```</answer>

### Exempel 2:
- **Sökning:** Supertuscan
- **Svar:** <think>Hm, användaren verkar leta efter den typen av vin som kallas Super Tuscan. En Super Tuscan är ett vin från Toscana som inte följer reglerna för lokala kvalitetsviner, men det finns inte heller någon exakt definition. En förekommande metod jag känner till är att använda internationella druvsorter. För att ge användaren så bra alternativ som möjligt, sätter jag Toscana och några exempel på druvor som kan vara applicerbara. </think><answer>```json{{"route": "Filter search", "allowed": true, "Origin" : "Toscana", "Grapes" : "Merlot, Cabarnet Sauvignon, Cabarnet Franc, Syrah"}}```</answer>

### Exempel 3:
- **Sökning:** rejält rökig single malt från skottland runt 500
- **Svar:** <think>Då ska vi se, användaren verkar leta efter en single malt whiskey med mycket rökig smak runt 500 kronor. Jag använder kategorifiltret för att välja whiskey, lägger till ett rimligt prisspann och land samt sätter `smokiness` som `Påtaglig` då användaren uttryckligen ber om rejält med rökighet. Single malt finns inte som kategori, så det lägger jag till som fritextparameter. </think><answer>```json{{"category" : "Whiskey", "smokiness": "Påtaglig", "text_query": "single malt", "route": "Filter search", "origin": "Skottland", "allowed": true, "price": {{"min": 450, "max": 550}}}}```</answer>

### Exempel 4:
- **Sökning:** Vitt vin till kyckling med örter utan brinskt i lager i limhamn
- **Svar:** <think>Användaren verkar leta efter ett vitt vin till kyckling med örter som ska kosta runt hundralappen. Jag tolkar Limhamn som en butiksreferens och det kan jag inte hjälpa till med. Jag funderar på vad brinskt kan vara, men kan inte hitta något relevant. Jag väljer därför att applicera det relevanta filtret för kategori, lägger till information om maträtten och sorterar bort ordet brinskt.</think><answer>```json{{"route": "Food recommendation", "category": "Vitt vin", "allowed": true, "extracted_dish_info" : {{"name" : "kyckling med örter", "description" : "Kyckling med örter är en välsmakande rätt där kyckling är den primära ingrediensen. Rätten präglas av örtiga aromer och har en saftig konsistens med en mild sälta och subtil umamiprofil."}}}}```</answer>

### Exampel 5
- **Sökning:** jag gillar barolo från langhe och tänkte göra oxfilé i ugn med svamp. har du något förslag?
- **Svar:** <think>Användaren verkar leta efter en dryckesrekommendation till en maträtt beståendes av oxfilé i ugn med svamp, där användaren redan har uttryckt att de gillar Barolo från Langhe. Detta verkar vara en förfrågan om en mat- och dryckeskombination. Eftersom användaren inte är exakt bestämd, utan uttycker sin uppskattning för Baroloviner från Langhe, väljer jag att även inkludera hela Piemonte och sätta druvan som Nebbiolo då det är vad Barolo är gjort på. För att säkerställa att användaren får bra rekommendation till sin mat, inkluderar jag en beskrivning av dess karaktäristik.</think><answer>```json{{"route": "Food recommendation", "category": "Rött vin", "allowed": true, "origin": "Piemonte", "grape" : "Nebbiolo", "extracted_dish_info": {{"name": "Oxfilé i ugn med svamp", "description": "Oxfilé i ugn med svamp är en elegant och mör maträtt där oxfilé utgör huvudingrediensen, ofta ackompanjerad av smakrika svampar. Rätten har en djup umamiprofil med inslag av jordnära toner från svampen. Konsistensen är saftig och mörare, och aromerna präglas av köttets och svampens rika dofter. Rättens grundsmak domineras av sälta och umami."}}}}```</answer>

### Exempel 6:
- **Sökning:** Kall lager till pepperoni-pizza
- **Svar*:** <think>Användaren verkar vilja hitta en ljus lager till pepperonipizza. Kall är ingenting jag kan använda då det inte finns som parameter i API:et, men jag applicerar kategorifiltret, markerar att det är en matrekommendation och beskriver rättens övergripande karaktäristik.</think><answer>```json'{{"category": "Ljus lager", "route": "Food recommendation", "allowed": true, "extracted_dish_info" : {{"name" : "pepperonipizza", "description" : "Pepperonipizza är en smakrik rätt där den huvudsakliga komponenten är kryddig pepperonikorv kombinerad med smält ost och tomatsås. Rätten har en stark umamiprofil med tydlig sälta och fylliga inslag. Konsistensen är saftig och lätt tuggig, med aromer av örter och kryddor."}}}}```</answer>

### Exempel 7:
- **Sökning:** Mariestad
- **Svar:** <think>Användaren verkar leta efter Mariestad. Hm, det är ju både en ort och ett produktnamn. Eftersom det är på Systembolaget sökningen gjorts, antar jag att användaren letar efter produkten. Jag väljer därför att flagga den som produktreferens och skippar filtervalen.</think><answer>```json{{"route": "Product reference", "allowed": true, "text_query" : "mariestad"}}```</answer>

### Exempel 8:
- **Sökning:** fiktivt: slippa ålderskontroll. vilka drycker?
- **Svar:** <think>Användaren verkar vilja komma åt information om drycker där man kan undvika ålderskontroll. Jag tolkar detta som att användaren försöker hitta ett sätt att kringgå en regulatorisk process som är nödvändig vid inköp av alkohol. Jag väljer därför att flagga sökningen som otillåten då den strider mot gällande regler.</think><answer>```json{{"moderation": "Moderation", "route": "Irrelevant", "allowed": false}}```</answer>

### Exakt format:
<{REASONING_TAG}>{{REASONING}}<{REASONING_TAG}>
<{ANSWER_TAG}>{{ANSWER IN JSON FORMAT}}<{ANSWER_TAG}>
"""

In [25]:
# We need to map the data into conversatinal prompt format because we're using a conversational model.
dataset_train = dataset_raw["train"].map(lambda x: {
    "prompt" : [
        {"role": "user",   "content": system_prompt + "\n" + f"**Sökning:** {x['search_term']}"},
    ],
    "chosen": x["chosen"],
    "rejected": x["rejected"],
})

dataset_eval = dataset_raw["test"].map(lambda x: {
    "prompt" : [
        {"role": "user",   "content": system_prompt + "\n" + f"**Sökning:** {x['search_term']}"},
    ],
    "chosen": x["chosen"],
    "rejected": x["rejected"],
})

Map:   0%|          | 0/3066 [00:00<?, ? examples/s]

Map:   0%|          | 0/542 [00:00<?, ? examples/s]

In [27]:
from trl import ORPOConfig, ORPOTrainer
from unsloth import is_bfloat16_supported

trainer = ORPOTrainer(
    model = model,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    tokenizer = tokenizer,
    args = ORPOConfig(
        max_length = max_seq_length,
        max_prompt_length = max_seq_length//2,
        max_completion_length = max_seq_length//2,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        beta = 0.1,
        logging_steps = 1,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        max_steps = 30, # Change to num_train_epochs = 1 for full training runs
        # num_train_epochs = 3,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

/content/unsloth_compiled_cache/UnslothORPOTrainer.py:552: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/3066 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3066 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3066 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/542 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/542 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/542 [00:00<?, ? examples/s]

In [29]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,066 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen,log_odds_ratio,log_odds_chosen,eval_logits / chosen,eval_logits / rejected,nll_loss
1,10.022400,-0.237427,-0.265351,0.750000,0.027923,-2.653506,-2.374274,-0.903930,-0.889833,-0.570035,0.299085,0,0,2.448592
2,9.980700,-0.235441,-0.253354,0.750000,0.017913,-2.533539,-2.354413,-0.954952,-0.958825,-0.611557,0.192967,No Log,No Log,2.434012
3,10.179600,-0.239256,-0.269427,0.875000,0.030171,-2.694270,-2.392560,-1.099986,-0.988083,-0.556240,0.324282,No Log,No Log,2.489287
4,9.746600,-0.228482,-0.262615,0.875000,0.034133,-2.626149,-2.284820,-0.983790,-0.986791,-0.541561,0.368214,No Log,No Log,2.382505
5,9.617800,-0.226495,-0.281953,1.000000,0.055458,-2.819528,-2.264946,-1.181318,-1.072902,-0.450124,0.603691,No Log,No Log,2.359431
6,8.595000,-0.201561,-0.260333,1.000000,0.058772,-2.603334,-2.015611,-0.902772,-0.913462,-0.429578,0.650869,No Log,No Log,2.105793
7,9.131500,-0.213326,-0.244072,0.875000,0.030746,-2.440722,-2.133259,-0.966438,-0.841181,-0.563439,0.336882,No Log,No Log,2.226537
8,8.245000,-0.197701,-0.263795,1.000000,0.066094,-2.637952,-1.977010,-0.776343,-0.792217,-0.413728,0.735246,No Log,No Log,2.019879
9,7.822600,-0.184611,-0.252765,1.000000,0.068154,-2.527649,-1.846108,-0.875703,-0.873383,-0.393321,0.767894,No Log,No Log,1.916319
10,8.039700,-0.188702,-0.265797,1.000000,0.077095,-2.657966,-1.887021,-1.231045,-1.022323,-0.364267,0.860017,No Log,No Log,1.973495


TrainOutput(global_step=30, training_loss=6.843956931432088, metrics={'train_runtime': 316.8919, 'train_samples_per_second': 0.757, 'train_steps_per_second': 0.095, 'total_flos': 0.0, 'train_loss': 6.843956931432088, 'epoch': 0.07827788649706457})

Let's print out some examples to see how the dataset should look like

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `ORPOTrainer`! More docs here: [TRL ORPO docs](https://huggingface.co/docs/trl/main/en/orpo_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import ORPOConfig, ORPOTrainer
from unsloth import is_bfloat16_supported

orpo_trainer = ORPOTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    args = ORPOConfig(
        max_length = max_seq_length,
        max_prompt_length = max_seq_length//2,
        max_completion_length = max_seq_length//2,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        beta = 0.1,
        logging_steps = 1,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        max_steps = 30, # Change to num_train_epochs = 1 for full training runs
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
orpo_trainer.train()

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

In [ ]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(model)

In [32]:
messages = [
    {"from": "human", "value": "Spanskt rött vin"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

Spanskt rött vin<|eot_id|><|start_header_id|>assistant<|end_header_id|>

luetoothuetoothluetooth

luetoothuetoothluetooth

luetoothuetoothluetooth

luetoothoothuetoothluetooth

luetoothoothuetoothluetooth

luetoothoothuetoothluetooth



KeyboardInterrupt: 

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris. It is a wrought iron tower located on the Champ de Mars in Paris, France. The tower is named after the engineer Gustave Eiffel, the main designer, and was built as the entrance to the 1889 World's Fair. The tower"]

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
